In [ ]:
# Import pacchetti
from bs4 import BeautifulSoup
import requests, lxml
import pandas as pd
import re
import numpy as np
from functools import reduce

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import dati di spotify
data = pd.read_csv('/content/drive/MyDrive/Data_man_Project/Data_Man_project/Data/artisti_sanremo_V2.csv', encoding = 'ISO-8859-1')
data.head()

,id_artist,nome,edizione
0,3xGlLcG9CUrs5MvFkSLOS5,Marco Mengoni,2013
1,61r4cE4aoOD50wPNhQ5fF7,Elio e le Storie Tese,2013
2,3ALm6zJLaJMWV0r89kuYtu,Modà,2013
3,5jcswtptZttxBwWKXbR5k5,Malika Ayane,2013
4,6hu9cUsJDFg2kUkAAjOepA,Raphael Gualazzi,2013


In [ ]:
data.nome=data.nome.apply(lambda x: x.split(', '))
print('pre explosion:', data.shape)
data= data.explode('nome').reset_index(drop=True)
print('post explosion:', data.shape)


pre explosion: (231, 3)
post explosion: (233, 3)


In [ ]:
data.loc[96,'id_artist']='4MR6tQyIrWK82b56cYPBDv'
data.loc[97,'id_artist']='6gXrMknarCxosAADyDHfme'
data.loc[98,'id_artist']='4KkD8gPbjRCdFybDN5wFNW'
data.loc[96:98]

,id_artist,nome,edizione
96,4MR6tQyIrWK82b56cYPBDv,Ornella Vanoni,2018
97,6gXrMknarCxosAADyDHfme,Bungaro,2018
98,4KkD8gPbjRCdFybDN5wFNW,Pacifico,2018


In [ ]:
# definizione variabili utili
nomi_artisti = data['nome'].to_list()

data['periodo_attivita'] = 0

In [ ]:
# SCRAPING
# ricerca periodo di attività dell'artista
# cerca nella pagina HTML il pattern " – " che porta a match del tipo '2005 – in attività'
for i in range(0, len(nomi_artisti)):
  query = 'https://www.google.com/search?q=' + nomi_artisti[i] + ' periodo di attività musicale'
  html = requests.get(query).text
  soup = BeautifulSoup(html, 'lxml')
  x = soup.find_all(string=re.compile(" – "))

  if len(x)!=0:
    x = x[0]
    data['periodo_attivita'][i] = x

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
data[15:25]

,id_artist,nome,edizione,periodo_attivita
15,6hu9cUsJDFg2kUkAAjOepA,Raphael Gualazzi,2014,2005 – in attività
16,0QJKELJZZuLAjqLOOixJm5,The Bloody Beetroots,2014,2006 – in attività
17,3uAIPzsjVYXpAek9PPxIXt,Renzo Rubino,2014,2011 – in attività
18,3TAYxk5QBcHunmb0qkeTn2,Francesco Renga,2014,1984 – in attività
19,62C5P1caRIK12ndTkzNJjA,Noemi,2014,–
20,6oHCbM8jtvJRe0BjKQ90mD,Perturbazione,2014,1988 – in attività
21,4vhlKamY3mdKkq6V6cChOJ,Cristiano De André,2014,1980 – in attività
22,15tvxTXbemwnKQq1aRoPpI,Frankie HI-NRG MC,2014,1992 – in attività
23,0dwEXtB6gceh7EO7tCsxhD,Giusy Ferreri,2014,1998 – In attività
24,16VwDz5nZgs3ZPEN5xmORY,Francesco Sarcina,2014,1999 – in attività


In [ ]:
# creazione nuovo dataset
df = data.copy()

In [ ]:
# pulizia colonna periodo_attivita
for i in range(0, len(df)):
  if df['periodo_attivita'][i] !=0:
    m = re.findall(r'\d+ – ', df['periodo_attivita'][i])
    if m:
      found = re.findall(r'\d+', m[0])
      df['periodo_attivita'][i] = found[0]

df[16:20]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,id_artist,nome,edizione,periodo_attivita
16,0QJKELJZZuLAjqLOOixJm5,The Bloody Beetroots,2014,2006
17,3uAIPzsjVYXpAek9PPxIXt,Renzo Rubino,2014,2011
18,3TAYxk5QBcHunmb0qkeTn2,Francesco Renga,2014,1984
19,62C5P1caRIK12ndTkzNJjA,Noemi,2014,–


In [ ]:
# inserimento NaN per artisti a cui non è stato trovato l'anno
df['periodo_attivita'] = df['periodo_attivita'].apply(str)

for i in range(0, len(df)):
  temp = re.search(r'\d+', df['periodo_attivita'][i])
  if temp is None or len(df['periodo_attivita'][i])>5:
    df['periodo_attivita'][i] = np.nan

df[16:20]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,id_artist,nome,edizione,periodo_attivita
16,0QJKELJZZuLAjqLOOixJm5,The Bloody Beetroots,2014,2006
17,3uAIPzsjVYXpAek9PPxIXt,Renzo Rubino,2014,2011
18,3TAYxk5QBcHunmb0qkeTn2,Francesco Renga,2014,1984
19,62C5P1caRIK12ndTkzNJjA,Noemi,2014,NaN


In [ ]:
# conteggio righe senza periodo di attività
count_scraping = df['periodo_attivita'].isna().sum()
print('Valori non trovati con metodo Scraping Google search: ', count_scraping)

Valori non trovati con metodo Scraping Google search:  8


In [ ]:
df[reduce(np.logical_and, (df['periodo_attivita'].isna() for c in df['periodo_attivita']))]

,id_artist,nome,edizione,periodo_attivita
19,62C5P1caRIK12ndTkzNJjA,Noemi,2014,NaN
54,62C5P1caRIK12ndTkzNJjA,Noemi,2016,NaN
109,62C5P1caRIK12ndTkzNJjA,Noemi,2018,NaN
121,2XpUprTohaksHpVjUtTvlx,Loredana Bertè,2019,NaN
130,2lkP8239tkwRjg5N7d2D0w,Frenetik&Orang3,2019,NaN
153,6gTrPTTb3XgiLt7GGcmf8j,Piero Pelù,2020,NaN
191,62C5P1caRIK12ndTkzNJjA,Noemi,2021,NaN
220,62C5P1caRIK12ndTkzNJjA,Noemi,2022,NaN


In [ ]:
df.shape

(233, 4)

In [ ]:
df

,id_artist,nome,edizione,periodo_attivita
0,3xGlLcG9CUrs5MvFkSLOS5,Marco Mengoni,2013,2009
1,61r4cE4aoOD50wPNhQ5fF7,Elio e le Storie Tese,2013,1980
2,3ALm6zJLaJMWV0r89kuYtu,Modà,2013,2002
3,5jcswtptZttxBwWKXbR5k5,Malika Ayane,2013,1995
4,6hu9cUsJDFg2kUkAAjOepA,Raphael Gualazzi,2013,2005
...,...,...,...,...
228,6GSNzwDDZ7M6bquJtrsH3g,Yuman,2022,2018
229,3J3PdLj6v1uAs1mWKpzZbX,Le Vibrazioni,2022,1999
230,0dwEXtB6gceh7EO7tCsxhD,Giusy Ferreri,2022,1998
231,6k8mwkKJKKjBILo7ypBspl,Ana Mena,2022,2006


In [ ]:
df.nome=df.nome.apply(lambda x: str.lower(x))

In [ ]:
df.to_csv('/content/drive/MyDrive/Data_man_Project/Data_Man_project/Data/artisti_final.csv', index = False)

codice di test (NON RUNNARE)

In [ ]:
# query = 'https://www.google.com/search?q=Enzo Avitabile+periodo+attività'
# html = requests.get(query).text
# soup = BeautifulSoup(html, 'lxml')
# x = soup.find_all(string=re.compile(" – "))

# x[0]

In [ ]:
# m = re.findall(r'\d+ – ', x[0])
# m

In [ ]:
# found = re.findall(r'\d+', m[0])
# found